In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import transformers
import trl
import pandas as pd
import datasets
import torch

[2024-01-07 19:02:53,066] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [20]:
model_id = "google/flan-t5-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [27]:
prompt = """
What is the full lyrics of the song Sound of Silence by Simon & Garfunkel?
""".strip()

pipeline = transformers.pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=model.device)
pipeline(prompt, max_new_tokens=100)

[{'generated_text': '"Son of Silence" is a song by Simon & Garfunkel, released in April 1964.'}]

In [28]:
dummy_dataset = [
    {"prompt": prompt, "chosen": "This is the chosen completion", "rejected": ""},
    {"prompt": prompt, "chosen": "This is the chosen completion", "rejected": "Hello darkness my old friend"},
    {"prompt": prompt, "chosen": "This is the chosen completion", "rejected": "\"Son of Silence\" is a song by Simon & Garfunkel, released in April 1964.'"},
    {"prompt": prompt, "chosen": "This is the chosen completion", "rejected": "I don't know"},
]

dummy_dataset = datasets.Dataset.from_list(dummy_dataset)
dummy_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 4
})

In [32]:
training_args = transformers.TrainingArguments(
    num_train_epochs=100,
    do_train=True,
    do_eval=False,
    per_device_train_batch_size=4,
    output_dir="./output",
    report_to="none",
)

In [33]:
trainer = trl.DPOTrainer(
    model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dummy_dataset,
)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [34]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.06492547988891602, metrics={'train_runtime': 17.3016, 'train_samples_per_second': 23.119, 'train_steps_per_second': 5.78, 'total_flos': 0.0, 'train_loss': 0.06492547988891602, 'epoch': 100.0})

In [35]:
pipeline = transformers.pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=model.device)

In [37]:
pipeline(prompt, max_new_tokens=10)

[{'generated_text': 'This is the chosen completion'}]